# Home heating system / temperature regulator
In this experiment we will present basic heating on/off system regulator with a hysteresis.
Even very modern thermostats like NEST are still primarly based on old ON/OFF regulaton method. 
![nest](img/nest.jpeg)

## Basic principle
User sets a desired temperature that is compared to measured temperature. If the measured temperature is < set temperature, we turn heater on, but if measured tempreature > set temperature we turn it off.
![on_off_control](img/on_off_control.png)

## Setup
- Red Pitaya STEMlab board + sensor shield
- Grove temperature sensor & LED diode for heater states indicator (optional relay that drives heater)
![temp_reg_setup](img/temp_reg_setup.jpg)

## Temperature sensor
The Grove - Temperature Sensor uses a Thermistor to detect the ambient temperature. The resistance of a thermistor will increase when the ambient temperature decreases. It's this characteristic that we use to calculate the ambient temperature. The detectable range of this sensor is -40 - 125ºC, and the accuracy is ±1.5ºC
Datasheet: [NCP18WF104F03RC](http://wiki.seeed.cc/Grove-Temperature_Sensor_V1.2/) 
![temp_sensor](img/temp_sensor.jpg)
![temp_sensor_sch](img/temp_sensor_sch.png)

In [ ]:
# Temperature vs. NTC resistance 
import math
import matplotlib.pyplot as plt
import time
from redpitaya.overlay.mercury import mercury as FPGA
overlay = FPGA()
ANALOG = FPGA.analog_in(0) # define which pin will be used for temperature sensor

def res_to_temp(Rntc):
    B = 4275; # B value of the thermistor
    return (1.0/(math.log10(Rntc/100000.0)/B+1/298.15)-273.15) #convert to temperature via datasheet 
temp = [ res_to_temp(R) for R in range(10000,300000,100)]
plt.plot(range(10000,300000,100), temp)
plt.xlabel('Rntc[ohm]')
plt.ylabel('Temperature[*C]')
plt.show()

## Reading temperature sensor

To read the temperature we need to read the voltage value and convert it to temperature.

In [ ]:
def temp():
    Va0=ANALOG.read() # read voltage of sensor
    R0 = 100000;       # R0 = 100k
    Rntc = 3.3/(Va0)-1.0 
    Rntc = 100000.0*Rntc # thermistor resistance
    return (res_to_temp(Rntc),Va0);
r=temp()

print("voltage={:.2f}V temp {:.2f}℃".format(r[1],r[0]) )

## Simple controll of LED

LEDs are controlled with GPIOs.

In [ ]:
GPIO = FPGA.gpio
import time

In [ ]:
gpio_o = GPIO('p', 0, "out")
gpio_o.write(True)
time.sleep(1)
gpio_o.write(False)

# Simple Home Heating

In [ ]:
import time

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

In [ ]:
colors = ('red', 'blue', 'green', 'orange')
legends = ('temperature', 'setTemp+hysteresis','setTemp-hysteresis', 'heater State')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="Home heating system", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.y_range=Range1d(20, 35)
p.yaxis.axis_label='temperature [℃]'

# get and explicit handle to update the next show cell with
# a simple ON/OFF regulator with hysteresis
target = show(p, notebook_handle=True)

In [ ]:
# a simple ON/OFF regulator with hysteresis
HYSTERESIS=0.5
setTemp=28
heaterState=0
chn=4

T = 1 # sampling period
HYST_HI=setTemp+HYSTERESIS
HYST_LO=setTemp-HYSTERESIS

x=[0]
y=[ [temp()[0]], [HYST_HI], [HYST_LO], [heaterState] ]
r = [p.line(x, y[ch], line_width=1, line_alpha=0.7, color=colors[ch],legend=legends[ch]) for ch in range(chn)]

# take 30 samples while simultaneously plotting the plot
for i in range(30):
    x.append(0) if i == 0 else x.append(x[-1]+T)
    temperature=temp()[0]
    
    # Hysteresis
    if (temperature < HYST_LO): #turn heating ON (LED)
        gpio_o.write(True)
        heaterState=1
    elif (temperature > HYST_HI): #turn heating OFF (LED)
        heaterState=0
        gpio_o.write(False)        
    
    r[0].data_source.data['y'].append(temperature) # store temperature
    r[1].data_source.data['y'].append(HYST_HI) # store set temperature + hysteresis
    r[2].data_source.data['y'].append(HYST_LO) # store set temperature - hysteresis
    r[3].data_source.data['y'].append(heaterState*22) # store heater state
      
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(T)